In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, Activation
import pickle
from keras.models import Model


Using TensorFlow backend.


In [104]:
#load and read csv file
data_module = pd.read_csv('Churn_Modelling.csv')
data_module.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [105]:
data_module.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [106]:
data_module.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [107]:
data_module.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [108]:
data_module.duplicated().sum()

0

In [109]:
x= data_module.iloc[:,3:13].values
y= data_module.iloc[:,13].values

In [110]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

labelencoder_x_1 =LabelEncoder()
x[:,1]=labelencoder_x_1.fit_transform(x[:,1])

labelencoder_x_2=LabelEncoder()
x[:,2]=labelencoder_x_2.fit_transform(x[:,2])

#oneHot Encoding
onehotencoder=OneHotEncoder(categorical_features=[1])
x= onehotencoder.fit_transform(x).toarray()
x=x[:,1:]

/home/admin172/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/admin172/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [111]:
print(x.shape,y.shape)

(10000, 11) (10000,)


In [112]:
# #spliting original dataset into train and test
# x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,random_state=0)
# print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)


# x_train,x_cv,y_train,y_cv = train_test_split(x_train,y_train,test_size=0.2,random_state=0)
# print("\n",x_train.shape,x_cv.shape,y_train.shape,y_cv.shape)

In [113]:
#split dataset into train and test
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)
x_train,x_cv,y_train,y_cv=train_test_split(x_train,y_train,test_size=0.30,random_state=0)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
print("\n",x_train.shape,x_cv.shape,y_train.shape,y_cv.shape)


(5600, 11) (2000, 11) (5600,) (2000,)

 (5600, 11) (2400, 11) (5600,) (2400,)


In [114]:
# Feature Scaling on independent variables
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train = sc.fit_transform(x_train)
x_cv = sc.transform(x_cv)
# x_train.shape,x_test.shape

In [115]:
file =open('test.pickle','wb')
pickle.dump(x_test,file)
pickle.dump(y_test,file)
pickle.dump(sc,file)
file.close()

In [116]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dense, Activation


In [117]:
classifier= Sequential()

In [118]:
# Adding the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(output_dim = 6,init = 'uniform',activation = 'relu',input_dim =11))


/home/admin172/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  


In [119]:
# Adding the second hidden layer, here no need to specify input dim
classifier.add(Dense(output_dim=6,init = 'uniform',activation = 'relu'))

/home/admin172/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  


In [120]:
# Adding the output layer, here no neeed to specify ip dim
classifier.add(Dense(output_dim=1, init='uniform',activation = 'sigmoid'))

/home/admin172/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [121]:
# Compiling the Artificial Neural Network
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])


In [122]:
# Fitting the Neural Network
classifier.fit(x_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
5600/5600 [==============================] - 1s 130us/step - loss: 0.5155 - acc: 0.7984
Epoch 2/100
5600/5600 [==============================] - 0s 80us/step - loss: 0.4286 - acc: 0.7993
Epoch 3/100
5600/5600 [==============================] - 0s 79us/step - loss: 0.4242 - acc: 0.7993
Epoch 4/100
5600/5600 [==============================] - 0s 79us/step - loss: 0.4209 - acc: 0.7993
Epoch 5/100
5600/5600 [==============================] - 0s 81us/step - loss: 0.4188 - acc: 0.7993
Epoch 6/100
5600/5600 [==============================] - 0s 77us/step - loss: 0.4165 - acc: 0.8141
Epoch 7/100
5600/5600 [==============================] - 0s 78us/step - loss: 0.4149 - acc: 0.8270
Epoch 8/100
5600/5600 [==============================] - 0s 75us/step - loss: 0.4136 - acc: 0.8266
Epoch 9/100
5600/5600 [==============================] - 0s 77us/step - loss: 0.4130 - acc: 0.8270
Epoch 10/100
5600/5600 [==============================] - 0s 77us/step - loss: 0.4121 - acc: 0.8300
Epoch 11

5600/5600 [==============================] - 0s 78us/step - loss: 0.3998 - acc: 0.8345
Epoch 84/100
5600/5600 [==============================] - 0s 81us/step - loss: 0.4001 - acc: 0.8329
Epoch 85/100
5600/5600 [==============================] - 0s 78us/step - loss: 0.4000 - acc: 0.8346
Epoch 86/100
5600/5600 [==============================] - 0s 77us/step - loss: 0.4001 - acc: 0.8359
Epoch 87/100
5600/5600 [==============================] - 0s 78us/step - loss: 0.4002 - acc: 0.8350
Epoch 88/100
5600/5600 [==============================] - 0s 75us/step - loss: 0.4001 - acc: 0.8341
Epoch 89/100
5600/5600 [==============================] - 0s 76us/step - loss: 0.4004 - acc: 0.8357
Epoch 90/100
5600/5600 [==============================] - 0s 75us/step - loss: 0.4000 - acc: 0.8341
Epoch 91/100
5600/5600 [==============================] - 0s 75us/step - loss: 0.4000 - acc: 0.8354
Epoch 92/100
5600/5600 [==============================] - 0s 75us/step - loss: 0.3994 - acc: 0.8345
Epoch 93/100


In [123]:
#predicting the  test set results

y_pred = classifier.predict(x_cv)
y_pred = (y_pred >0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [ True]
 [False]
 [False]]


In [124]:
accuracy= accuracy_score(y_cv,y_pred)*100
print(accuracy)

83.66666666666667


In [127]:
#calculate confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_cv,y_pred)
cm

array([[1847,   45],
       [ 347,  161]])

In [128]:
#create a pickle file
file =open("train.pickle","wb")
pickle.dump(classifier,file)
file.close()